In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

In [3]:
# read in the data from the CSV file
data = pd.read_csv('video_games_dataset.csv')

# define the features to use for clustering
features = data[['esrb_rating', 'platform', 'genres']]

# convert categorical features to numerical using one-hot encoding
features = pd.get_dummies(features, columns=['esrb_rating', 'platform', 'genres'])

# use k-means clustering to group similar games together
num_clusters = 20  # you can adjust this to your liking
km = KMeans(n_clusters=num_clusters, n_init=10)
km.fit(features)

# add the cluster labels to our dataframe
data['cluster'] = km.labels_

In [7]:
# define a function to get recommended games based on a given game title
def get_recommendations(input_title):
    game = data[data['title'] == input_title]
    if len(game) == 0:
        return []
    cluster = game.iloc[0]['cluster']
    similar_games = data[data['cluster'] == cluster]
    similar_games = similar_games[similar_games['title'] != input_title]
    return similar_games.sort_values(['meta_score', 'user_score'], ascending=False).head(5)


# create a function to handle the search button click event
def on_search_button_clicked(b):
    with output:
        clear_output()
        
        title = game_selector.value
        input_game_index = data[data['title'] == title].index[0]
        input_game_platform = data.loc[input_game_index, 'platform']
        recommendations = get_recommendations(title)
    
        if recommendations.empty:
            print('Sorry, we could not find any recommendations for that game.')
        else:
            results_header = widgets.HTML(value=f"<h3>Recommended games for <u><i>{title}</i></u>:</h3>", layout=widgets.Layout(width='750px'))
        
            table_html = f'<table>\
                            <tr>\
                                <th>Title</th>\
                                <th>ESRB Rating</th>\
                                <th>Platform</th>\
                                <th>Meta Score</th>\
                                <th>Genres</th>\
                            </tr>'
        
            for i, row in recommendations.iterrows():
                table_html += f'<tr>\
                                    <td>{row["title"]}</td>\
                                    <td>{row["esrb_rating"]}</td>\
                                    <td>{row["platform"]}</td>\
                                    <td>{row["meta_score"]}</td>\
                                    <td>{row["genres"]}</td>\
                                </tr>'
            
            table_html += '</table>'
        
            # display results header and recommendation table
            display(results_header)
            display(HTML(table_html))
            
            bar_chart_title = widgets.HTML(value=f"<h4>Recommended games for <u><i>{title}</i></u>, organized by Meta Score and User Score:</h4>", layout=widgets.Layout(width='750px'))
            #score_definitions = widgets.HTML(value="<h6><b>Meta Score - (0 - 100)</b> is a weighted average of reviews from top critics and publications.<br><b>User Score - (0 - 10)</b> is a weighted average of reviews from actual gamers.</h6>")
        
            # plot a bar chart of the top 5 recommended games by meta score
            top_recommendations1 = recommendations.sort_values('meta_score', ascending=False).head(5)
            fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(7,4))

            # plot the first bar chart in the left subplot
            ax1.bar(top_recommendations1['title'], top_recommendations1['meta_score'], color='blue')

            ax1.set_ylabel('Meta Score')
            ax1.tick_params(axis='x', rotation=90)

            # plot the second bar chart in the right subplot
            top_recommendations2 = recommendations.sort_values('user_score', ascending=False).head(5)
            ax2.bar(top_recommendations2['title'], top_recommendations2['user_score'], color='purple')

            ax2.set_ylabel('User Score')
            ax2.tick_params(axis='x', rotation=90)

            # adjust the spacing between subplots
            plt.subplots_adjust(wspace=0.75)

            #display bar_chart_header with bar charts
            display(bar_chart_title)
            plt.show()
            #display(score_definitions)
            
            pie_chart_title = widgets.HTML(value=f"<h4>ESRB Ratings & Platforms for <u><i>{title}</i></u>-like Game Recommendations:</h4>", layout=widgets.Layout(width='750px'))
            #ESRB_definition = widgets.HTML(value="<h6><b>ESRB Ratings</b> are a standardized content rating system used for video games, providing guidance on the age appropriateness and content of the game.</h6>", layout=widgets.Layout(width='750px'))
            
            # count the frequency of each esrb_rating and platform
            esrb_rating_counts = recommendations['esrb_rating'].value_counts()
            platform_counts = recommendations['platform'].value_counts()

            # create the first pie chart
            fig, ax = plt.subplots(1, 2, figsize=(7, 4))

            ax[0].pie(esrb_rating_counts, labels=esrb_rating_counts.index)
            ax[0].set_title('ESRB Rating')

            # create the second pie chart
            ax[1].pie(platform_counts, labels=platform_counts.index)
            ax[1].set_title('Platform')
            
            # adjust the spacing between subplots
            plt.subplots_adjust(wspace=0.5)

            # display the two pie charts side by side
            display(pie_chart_title)
            plt.show()
            #display(ESRB_definition)


image1 = open("egs_logo.jpg", "rb")
everyn_gaming_store_logo = image1.read()

logo_headline = widgets.HBox(
    [widgets.Image(value=everyn_gaming_store_logo, format='jpg', width='200')],
    layout=widgets.Layout(justify_content='center', width='750px')
)

# create the dropdown widget with the list of game titles
game_titles = data['title'].unique()
game_selector = widgets.Dropdown(options=sorted(game_titles))

# create the search button
search_button = widgets.Button(description="Search")
output = widgets.Output()

# attach the function to the search button click event
search_button.on_click(on_search_button_clicked)

# Create the Page Header

text_0 = widgets.HTML(value="<h1 style='text-align: center;'>Nintendo Video Game Recommendation App</h1>", layout=widgets.Layout(width='750px'))
text_1 = widgets.HTML(value="<h3 style='text-align: center;'><i>Experience gaming at its best with personalized game recommendations tailored exclusively for you, elevating your gaming journey and boosting your confidence in every purchase decision.</i></h3>", layout=widgets.Layout(width='750px'))
page_header = widgets.VBox([text_0, text_1], justify_content='center')

# Create the Search Section
text_3 = widgets.HTML(value="<h3 style='text-align: center;'>Select your preferred Nintendo video game:</h3>", layout=widgets.Layout(width='750px'))
page_search_components = widgets.HBox([game_selector, search_button], layout=widgets.Layout(justify_content='center', width='750px'))
page_search = widgets.VBox([text_3, page_search_components], justify_content='center')

# Create and Display the page
page = widgets.VBox([logo_headline, page_header, page_search, output], layout=widgets.Layout(justify_content='center', width='900px'))
display(page)